# BIOINF 575 Project 1
_______
## Parts 1-3

In [1]:
#set up sql stuff
from sqlite3 import connect
import pandas as pd 

connection = connect("project_b575.sqlite")
cursor = connection.cursor()

def display_results(cursor):
    data = cursor.fetchall()
    header = [t[0] for t in cursor.description]
    res_df = pd.DataFrame(data, columns = header)
    return res_df

In [2]:
#create a table for the 23andMe data
create_23andme = '''
CREATE TABLE IF NOT EXISTS df_23andme (
    variant_id INTERGER PRIMARY KEY,
    chr TEXT,
    position INTERGER,
    dbSNP_ID TEXT,
    allele_23andme TEXT,
    FOREIGN KEY (dbSNP_ID) REFERENCES pharmGKB (variant)
)
'''

cursor.execute(create_23andme)

In [3]:
#create a table for the PharmGKB data
create_pharmGKB = '''
CREATE TABLE IF NOT EXISTS pharmGKB (
    row_id INTERGER PRIMARY KEY,
    var_ann_id INTERGER,
    variant TEXT,
    gene_sym TEXT,
    drug TEXT,
    PMID TEXT,
    phenotype_cat TEXT,
    significance TEXT,
    notes TEXT,
    sentence TEXT,
    allele_pharmGKB TEXT
)
'''
cursor.execute(create_pharmGKB)

In [4]:
#create index 
create_idx = '''
CREATE INDEX SNP_idx
ON pharmGKB (variant)
'''
cursor.execute(create_idx)

In [5]:
#add the 23andMe data to the table
insert_variants = '''
INSERT INTO df_23andme
VALUES (?, ?, ?, ?, ?)
'''

variant_id = 0 
with open('23andme_v5_hg19_ref.txt') as variant_file:
    for line in variant_file:
        variant_id = variant_id + 1
        line_list = line.strip().split('\t')
        chromosome = line_list[0]
        position = int(line_list[1])
        SNP = line_list[2]
        allele = line_list[3]
        
        variant_row = [variant_id, chromosome, position, SNP, allele]
        cursor.execute(insert_variants, variant_row)

In [6]:
#add the PharmGKB data to the table
insert_pharmGKB = '''
INSERT INTO pharmGKB
VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
'''

row_id = 0 
with open('var_drug_ann.tsv') as drug_file:
    header_line = drug_file.readline()
    for line in drug_file: 
        row_id = row_id + 1
        if row_id == 5487:
            continue
        line_list = line.strip().split('\t')
        var_ann_id = int(line_list[0])
        variant = line_list[1]
        gene_sym = line_list[2]
        drug = line_list[3]
        pmid = line_list[4]
        phenotype_cat = line_list[5]
        significance = line_list[6]
        notes = line_list[7]
        sentence = line_list[8]
        allele = line_list[9]
        
        drug_row = [row_id, var_ann_id, variant, gene_sym, drug, pmid, phenotype_cat, significance, notes, sentence, allele]
        cursor.execute(insert_pharmGKB, drug_row)
        

In [7]:
#merge tables based on dbSNP_ID
sql = '''
SELECT variant, gene_sym, drug, PMID, phenotype_cat, significance, notes, sentence, allele_pharmGKB, allele_23andme
FROM pharmGKB
    INNER JOIN df_23andme on pharmGKB.variant = df_23andme.dbSNP_id
'''

cursor.execute(sql)
df_merge = display_results(cursor)

In [8]:
#filter the table for significant associations affecting efficiacy
#df_merge is the merged dataframe of the two files
df_filtered = df_merge[(df_merge.significance == 'yes') & (df_merge.phenotype_cat == 'Efficacy')]
df_filtered = df_filtered.drop(columns = ['phenotype_cat', 'significance', 'PMID'])
df_filtered.head(10)

,variant,gene_sym,drug,notes,sentence,allele_pharmGKB,allele_23andme
8,rs1801131,MTHFR,oxaliplatin,Statistics given as trend for increased overal...,Genotypes GG + GT are associated with increase...,GG + GT,T
9,rs1801131,MTHFR,"Vitamin B-complex, Incl. Combinations",Please note that allele has been complemented ...,Allele G is associated with increased response...,G,T
11,rs1801131,MTHFR,methotrexate,Alleles given as C and A. Efficacy of treatmen...,Genotype GT is associated with decreased respo...,GT,T
14,rs1801131,MTHFR,methotrexate,Patients with the GG genotype had better overa...,Genotype GG is associated with increased respo...,GG,T
15,rs1801131,MTHFR,methotrexate,Patients with the TT genotype had significantl...,Genotype TT is associated with decreased respo...,TT,T
30,rs1801131,MTHFR,methotrexate,,Genotype GG is associated with decreased respo...,GG,T
31,rs1801131,MTHFR,methotrexate,,Genotypes GT + TT is associated with increased...,GT + TT,T
34,rs1801131,"C1orf167, CLCN6, MTHFR",methotrexate,Non-response was classified by DAS28 >3.2 at t...,Genotype TT is associated with decreased respo...,TT,T
36,rs1801131,MTHFR,"fluorouracil, leucovorin, oxaliplatin",,Allele G is associated with increased response...,G,T
40,rs1801131,MTHFR,benazepril,"When in a haplotype with rs1801133 allele G, a...",Allele G is associated with decreased response...,G,T


In [12]:
#save the data
#df_filtered is the file to save
df_filtered.to_csv('23andme_PharmGKB_map.tsv', sep = '\t', header = True, index = False)

__________
## <u> Discussion Questions <u>

__________
## <u> Discussion Questions <u>

### Question 1: What was your biggest challenge in this project?
The biggest challenge with this project's code itself was 
### Question 2: What did you learn while working on this project?
This project helped reinforce my understanding and practice of data management through SQL and Pandas. It also helped me practice my coding abilities in Python. Additionally, the data itself was really interesting, and I didn't fully appreaciate the impact that genetic variation plays on drug efficacy. 
### Question 3: If you had more time on the project what other question(s) would you like to answer?
